In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import os
import cv2
import pydicom
import timeit
import seaborn as sns
plt.style.use('seaborn-talk')
import glob
from pathlib import Path
from tqdm import tqdm
from matplotlib import animation, rc
import joblib
import difflib


In [ ]:
def normalize_by_img_max(img):
#     print(img.min(), img.max(), img.dtype)
    img_max = max(1, img.max())
    return (img / img_max * 255).astype('uint8')

def read_dcm(dcm_name, normalize=True):
    np_img = pydicom.read_file(dcm_name).pixel_array
    if normalize:
        np_img = normalize_by_img_max(np_img)
    return np_img

read_dcm('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-273.dcm', normalize=False)

In [ ]:
def compare_two_dicom_headers(file1, file2):
    datasets = tuple([pydicom.dcmread(filename, force=True)
                  for filename in (file1, file2)])

    rep = []
    for dataset in datasets:
        lines = str(dataset).split("\n")
        lines = [line + "\n" for line in lines]  # add the newline to end
        rep.append(lines)


    diff = difflib.Differ()
    for line in diff.compare(rep[0], rep[1]):
        if line[0] != "?" and line[0] in ['-', '+']:
            print(line)
    
file_name1 = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-273.dcm'
file_name2 = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-272.dcm'

compare_two_dicom_headers(file_name1, file_name2)

In [ ]:
pydicom.read_file('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00031/T1w/Image-33.dcm')

# <h1> Resample volume  1x1x1mm volumes

In [ ]:
data_path = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00031/T1w"
g = glob.glob(data_path + '/*.dcm')

# Print out the first 5 file names to verify we're in the right folder.
print ("Total of %d DICOM images.\nFirst 5 filenames:" % len(g))
print(g[:5])

In [ ]:
def load_scan(path):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))

    slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
    print(slice_thickness)
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 1
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

id=0
patient = load_scan(data_path)
imgs = get_pixels_hu(patient)
print(imgs.shape)
np.save("fullimages_%d.npy" % (id), imgs)

In [ ]:
file_used="fullimages_%d.npy" % id

imgs_to_process = np.load(file_used).astype(np.float64) 

plt.hist(imgs_to_process.flatten(), bins=50, color='c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
print("Slice Thickness: ", patient[0].SliceThickness)
print("Pixel Spacing (row, col):",  (patient[0].PixelSpacing[0], patient[0].PixelSpacing[1]))

In [ ]:
import scipy
id = 0
imgs_to_process = np.load('fullimages_{}.npy'.format(id))
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = map(float, ([scan[0].SliceThickness] + list(scan[0].PixelSpacing)))
    spacing = np.array(list(spacing))
    resize_factor = spacing / new_spacing
    print(resize_factor)
    new_real_shape = image.shape * resize_factor
    print(new_real_shape)
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)
    
    return image, new_spacing

print("Shape before resampling\t", imgs_to_process.shape)
imgs_after_resamp, spacing = resample(imgs_to_process, patient, [1,1,1])
print("Shape after resampling\t", imgs_after_resamp.shape)

In [ ]:
spacing

In [ ]:
def plot_volume_axis(volume):
    fig = plt.figure(figsize=(20, 20))
    ax1 = fig.add_subplot(1, 3, 1)
    ax1.imshow(volume[volume.shape[0]//2], cmap='gray')
    
    ax2 = fig.add_subplot(1, 3, 2)
    ax2.imshow(volume[:, volume.shape[1]//2], cmap='gray')
    
    ax3 = fig.add_subplot(1, 3, 3)
    ax3.imshow(volume[:, :, volume.shape[2]//2], cmap='gray')
    fig.show()

plot_volume_axis(imgs_after_resamp)

In [ ]:
from skimage import measure
import plotly
import plotly.express as px
from plotly.offline import  iplot
from plotly.tools import FigureFactory as FF


def make_mesh(image, threshold=-300, step_size=1):

    print("Transposing surface")
    p = image.transpose(2,1,0)
    
    print("Calculating surface")
    verts, faces, norm, val = measure.marching_cubes(p, threshold, step_size=step_size, allow_degenerate=True) 
    return verts, faces

def plotly_3d(verts, faces):
    x,y,z = zip(*verts) 
    
    print("Drawing")
    # Make the colormap single color since the axes are positional not intensity. 
#    colormap=['rgb(255,105,180)','rgb(255,255,51)','rgb(0,191,255)']
    colormap= px.colors.sequential.Brwnyl
    backgroundcolor = 'slategray'
    fig = FF.create_trisurf(x=x,
                        y=y, 
                        z=z, 
                        plot_edges=False,
                        colormap=colormap,
                        simplices=faces,
                        backgroundcolor=backgroundcolor,
                        title="Interactive Visualization")
    iplot(fig)

    
v, f = make_mesh(imgs_after_resamp, threshold=10, step_size=10)
plotly_3d(v, f)


<h1> Volume preprocessing Pipeline

In [ ]:
def get_image_plane(data):
    x1, y1, _, x2, y2, _ = [round(j) for j in data.ImageOrientationPatient]
    cords = [x1, y1, x2, y2]

    if cords == [1, 0, 0, 0]:
        return 'Coronal'
    elif cords == [1, 0, 0, 1]:
        return 'Axial'
    elif cords == [0, 1, 0, 0]:
        return 'Sagittal'
    else:
        return 'Unknown'
    
dcm_img = pydicom.read_file('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00031/T1w/Image-22.dcm')
plane = get_image_plane(dcm_img)
plt.imshow(dcm_img.pixel_array, cmap='gray')
plane

In [ ]:
def get_volume(study_dir):
    imgs = []
    dcm_dir = Path(study_dir)
    dcm_paths = sorted(dcm_dir.glob("*.dcm"), key=lambda x: int(x.stem.split("-")[-1]))
    positions = []
    
    for dcm_path in dcm_paths:
        img = pydicom.dcmread(str(dcm_path))
        imgs.append(img.pixel_array)
        positions.append(img.ImagePositionPatient)
        
    plane = get_image_plane(img)
    volume = np.stack(imgs)
    
    # reorder planes if needed and rotate volume
    if plane == "Coronal":
        if positions[0][1] < positions[-1][1]:
            volume = volume[::-1]
            print(f"{study_id} {scan_type} {plane} reordered")
        volume = volume.transpose((1, 0, 2))
    elif plane == "Sagittal":
        if positions[0][0] < positions[-1][0]:
            volume = volume[::-1]
            print(f"{study_id} {scan_type} {plane} reordered")
        volume = volume.transpose((1, 2, 0))
        volume = np.rot90(volume, 2, axes=(1, 2))
    elif plane == "Axial":
        if positions[0][2] > positions[-1][2]:
            volume = volume[::-1]
            print(f"{study_id} {scan_type} {plane} reordered")
        volume = np.rot90(volume, 2)
    else:
        raise ValueError(f"Unknown plane {plane}")
    return volume, plane

In [ ]:
from skimage.transform import rescale, resize, downscale_local_mean


def calc_padding_inds(keep, padding):
    s_edge_ind = np.argmax(keep)
    e_edge_ind = np.argmax(keep[::-1])
    keep_s = max(0, s_edge_ind-padding)
    keep_e = min(len(keep)-e_edge_ind+padding, len(keep))
    return keep_s, keep_e

def crop_volume(volume, padding=40):
    if volume.sum() == 0:
        return volume
    keep = (volume.mean(axis=(0, 1)) > 0)
    keep_s, keep_e = calc_padding_inds(keep, padding)
    volume = volume[:, :, keep_s:keep_e]
    
    
    keep = (volume.mean(axis=(0, 2)) > 0)
    keep_s, keep_e = calc_padding_inds(keep, padding)
    volume = volume[:, keep_s:keep_e]
    
    
    keep = (volume.mean(axis=(1, 2)) > 0)
    keep_s, keep_e = calc_padding_inds(keep, padding)

    volume = volume[keep_s:keep_e]
    return volume

def resize_volume(volume, sz=128, interpolation=cv2.INTER_LINEAR):
    output = np.zeros((sz, sz, sz), dtype=np.float32)

    if np.argmax(volume.shape) == 0:
        for i, s in enumerate(np.linspace(0, volume.shape[0] - 1, sz)):
            output[i] = cv2.resize(volume[int(s)], (sz, sz), interpolation=interpolation)

    elif np.argmax(volume.shape) == 1:
        for i, s in enumerate(np.linspace(0, volume.shape[1] - 1, sz)):
            output[:, i] = cv2.resize(volume[:, int(s)], (sz, sz), interpolation=interpolation)

    elif np.argmax(volume.shape) == 2:
        for i, s in enumerate(np.linspace(0, volume.shape[2] - 1, sz)):
            output[:, :, i] = cv2.resize(volume[:, :, int(s)], (sz, sz), interpolation=interpolation)

    return output

In [ ]:
!pip install --upgrade https://github.com/VincentStimper/mclahe/archive/numpy.zip
from mclahe import mclahe

<h2><b><i> Pipeline </i> </b></h2>

In [ ]:
volume, _ = get_volume("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00045/FLAIR")
print(volume.shape, volume.max())

volume = crop_volume(volume, padding=5)
volume = resize_volume(volume)

print(volume.shape, volume.max())

volume = mclahe(volume, n_bins=64)
print(volume.shape, volume.max())

plot_volume_axis(volume)
v, f = make_mesh(volume, threshold=0, step_size=2)
plotly_3d(v, f)
